# Check info

In [ ]:
import pandas as pd

# goodreads_books info (first 100k samples)
chunk_iterator = pd.read_json(
    '/content/drive/MyDrive/data/goodreads_books.json',
    lines=True,         # Format with one JSON object per line
    encoding='latin-1', # Prevent encoding issues
    chunksize=200_000   # Read in chunks of 100k
)

df_goodreads_books = next(chunk_iterator)
print(df_goodreads_books.head())
print(df_goodreads_books.info())

In [ ]:
# goodreads_book_authors info (first 100k samples)
chunk_iterator = pd.read_json(
    '/content/drive/MyDrive/data/goodreads_book_authors.json',
    lines=True,         # Format with one JSON object per line
    encoding='latin-1', # Prevent encoding issues
    chunksize=200_000   # Read in chunks of 100k
)

df_goodreads_book_authors = next(chunk_iterator)
print(df_goodreads_book_authors.head())
print(df_goodreads_book_authors.info())

In [ ]:
# goodreads_reviews_dedup info (first 100k samples)
chunk_iterator = pd.read_json(
    '/content/drive/MyDrive/data/goodreads_reviews_dedup.json',
    lines=True,         # Format with one JSON object per line
    encoding='latin-1', # Prevent encoding issues
    chunksize=200_000   # Read in chunks of 100k
)

df_goodreads_reviews = next(chunk_iterator)
print(df_goodreads_reviews.head())
print(df_goodreads_reviews.info())

In [ ]:
# Books_5.json info (first 100k samples)


chunk_iterator = pd.read_json(
    '/content/drive/MyDrive/data/Books_5.json',
    lines=True,           # Format with one JSON object per line
    encoding='latin-1',   # Prevent encoding issues
    chunksize=300_000     # Read in chunks of 100k
)

df_books_5 = next(chunk_iterator)
print(df_books_5.head())
print(df_books_5.info())

In [ ]:
# meta_Books.json info (first 200k samples)
import ast  # Needed to convert text with single quotes ('') into a dictionary

data_list = []

# Instead of pd.read_json, read the file line by line manually
with open('/content/drive/MyDrive/data/meta_Books.json', 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i >= 200_000:
            break

        # Use ast.literal_eval to convert text with single quotes into a dictionary
        if line.strip(): # Only execute if the line is not empty
            data_list.append(ast.literal_eval(line.strip()))

# Convert the collected list of dictionaries into a DataFrame.
df_meta_books = pd.DataFrame(data_list)
print(df_meta_books.head())
print(df_meta_books.info())

# Data Generation

In [ ]:
import pandas as pd
import numpy as np
import ast
import os

# --- Helper Function ---
def extract_first_author_id(authors_list):
    """
    Function to extract 'author_id' from Goodreads 'authors' column
    """
    try:
        if isinstance(authors_list, list) and len(authors_list) > 0:
            first_author = authors_list[0]
            if isinstance(first_author, dict):
                return first_author.get('author_id')
    except Exception as e:
        pass
    return None

In [ ]:
# --- Define File Paths ---
BASE_PATH = '/content/drive/MyDrive/data/'
path_d1_books   = os.path.join(BASE_PATH, 'goodreads_books.json')
path_d1_authors = os.path.join(BASE_PATH, 'goodreads_book_authors.json')
path_d1_reviews = os.path.join(BASE_PATH, 'goodreads_reviews_dedup.json')
path_d2_meta    = os.path.join(BASE_PATH, 'meta_Books.json')
path_d2_reviews = os.path.join(BASE_PATH, 'Books_5.json')

FINAL_GOODREADS_CSV = os.path.join(BASE_PATH, 'df_goodreads_final.csv')
FINAL_AMAZON_CSV    = os.path.join(BASE_PATH, 'df_amazon_final.csv')

# --- Settings ---
TARGET_MASTER_COUNT = 70_000 # Number of common books. Limited to 70k due to large data size
CHUNK_SIZE = 100_000 # Amount of data to load into RAM at once

In [ ]:
# --- 1. Load Author Mapping Table ---
print("\n--- Loading Goodreads Author (D1 Authors) Mapping Table ---")
df_authors_full = pd.read_json(path_d1_authors, lines=True, encoding='latin-1')
df_authors_full['Book-Author-ID'] = df_authors_full['author_id'].astype(str)
author_id_to_name_map = df_authors_full.set_index('Book-Author-ID')['name'].to_dict() # Convert to a dictionary of {'author_id': 'name'} format
print(f"  > Author mapping table for {len(author_id_to_name_map)} authors created successfully.")

In [ ]:
# --- 2. Scan Amazon Meta ISBN and Create Set ---
# Scan the entire D2(Amazon) metadata file to collect all unique book ISBNs ('asin') present in Amazon.
print("\n--- Scanning Amazon Meta (D2 Meta) ISBN 'Set' ---")
amazon_isbn_set = set()
with open(path_d2_meta, 'r', encoding='utf-8', errors='ignore') as f:
    for i, line in enumerate(f):
        if line.strip():
            try:
                book_dict = ast.literal_eval(line.strip())
                if 'asin' in book_dict and book_dict['asin']:
                    amazon_isbn_set.add(book_dict['asin']) # Add asin (ISBN) to the set
            except:
                continue # Skip lines with parsing errors
print(f"  > Collected {len(amazon_isbn_set)} unique Amazon(D2) ISBNs.")

In [ ]:
# --- 3. Scan Goodreads Books and Extract Common Book Candidates ---
# Scan the D1(Goodreads) book list (tens of millions) and find 70,000 common books whose ISBN exists in the 'amazon_isbn_set' created in step 2.
print(f"\n--- Step 3: Scanning D1(Goodreads Books) and extracting {TARGET_MASTER_COUNT} common book candidates based on Amazon Set ---")
master_book_candidates = [] # List to store temporary candidates
found_count = 0             # Counter for the number found so far

# Read in chunks
d1_books_iterator = pd.read_json(
    path_d1_books,
    lines=True,
    encoding='latin-1',
    chunksize=CHUNK_SIZE
)

for chunk in d1_books_iterator:
    chunk_filtered = chunk[chunk['isbn'].isin(amazon_isbn_set)].copy()

    # If common books are found, slice the required number so as not to exceed the count
    if not chunk_filtered.empty:
        needed = TARGET_MASTER_COUNT - found_count
        if len(chunk_filtered) > needed:
            chunk_filtered = chunk_filtered.iloc[:needed]

        chunk_filtered['Book-Author-ID'] = chunk_filtered['authors'].apply(extract_first_author_id)

        # Select only the final columns to be used
        d1_master_cols = ['book_id', 'isbn', 'title', 'publisher', 'Book-Author-ID']
        d1_cols_final = [col for col in d1_master_cols if col in chunk_filtered.columns]
        chunk_ready = chunk_filtered[d1_cols_final].copy()

        # Add to the master_book_candidates list
        master_book_candidates.append(chunk_ready)
        found_count += len(chunk_ready)
        print(f"  ... {found_count} / {TARGET_MASTER_COUNT} found") # Print progress

    if found_count >= TARGET_MASTER_COUNT:
        print(f"  > Target of {TARGET_MASTER_COUNT} reached. Stopping scan.") # Print progress
        break

In [ ]:
# --- 4. Complete df_master_books ---
print("\n--- Step 4: Completing df_master_books (Common Books) ---")
if master_book_candidates:
    df_master_books = pd.concat(master_book_candidates, ignore_index=True)
    df_master_books = df_master_books.head(TARGET_MASTER_COUNT)

    # Change author id to author name
    df_master_books['Book-Author'] = df_master_books['Book-Author-ID'].map(author_id_to_name_map)
    # Standardize column names
    df_master_books = df_master_books.rename(columns={
        'isbn': 'ISBN', 'title': 'Book-Title', 'publisher': 'Publisher'
    })
    final_master_cols = ['book_id', 'ISBN', 'Book-Title', 'Publisher', 'Book-Author']
    df_master_books = df_master_books[final_master_cols]
    print(f"  > Final {len(df_master_books)} master books confirmed. (Loaded into memory)") # [Print progress]
else:
    print("  > Warning: No common books were found.") # [Print progress]
    df_master_books = pd.DataFrame(columns=['book_id', 'ISBN', 'Book-Title', 'Publisher', 'Book-Author'])

# Define sets for filtering
master_book_id_set_final = set(df_master_books['book_id'])
master_isbn_set_final = set(df_master_books['ISBN'])
# Define final csv column order
final_columns = ['User-ID', 'ISBN', 'Book-Rating', 'Book-Title', 'Book-Author', 'Publisher', 'Review']

In [ ]:
# --- 5. Create Final Goodreads DataFrame (Write directly to CSV) ---
# Scan all D1(Goodreads) reviews, select only those corresponding to df_master_books, and save
print(f"\n--- Step 5: Scanning D1(Goodreads Reviews) and saving to CSV -> {FINAL_GOODREADS_CSV} ---")
is_first_chunk = True
if master_book_id_set_final:
    d1_reviews_iterator = pd.read_json(
        path_d1_reviews, lines=True, encoding='latin-1', chunksize=CHUNK_SIZE
    )
    for i, chunk in enumerate(d1_reviews_iterator):
        print(f"   > Processing D1 Reviews chunk {i+1}...") # [Print progress]
        chunk_filtered = chunk[chunk['book_id'].isin(master_book_id_set_final)].copy()

        if not chunk_filtered.empty:
            # Standardize column names
            d1_rating_cols = ['user_id', 'book_id', 'rating', 'review_text']
            chunk_ready = chunk_filtered[d1_rating_cols].copy()
            chunk_ready = chunk_ready.rename(columns={
                'user_id': 'User-ID', 'rating': 'Book-Rating', 'review_text': 'Review'
            })

            # Merge reviews and master_books based on book_id
            df_goodreads_chunk_final = pd.merge(
                chunk_ready, df_master_books, on='book_id', how='inner'
            )
            df_goodreads_chunk_final = df_goodreads_chunk_final[final_columns]

            # First chunk: write mode creates a new file and includes header
            if is_first_chunk:
                df_goodreads_chunk_final.to_csv(FINAL_GOODREADS_CSV, mode='w', header=True, index=False, encoding='utf-8')
                is_first_chunk = False

            # Second chunk: append mode adds data only
            else:
                df_goodreads_chunk_final.to_csv(FINAL_GOODREADS_CSV, mode='a', header=False, index=False, encoding='utf-8')
else:
    print("   > Master book list (book_id) is empty. Skipping Goodreads reviews scan.") # [Print progress]

print(f"   > [df_goodreads_final] save complete: {FINAL_GOODREADS_CSV}") # [Print progress]

In [ ]:
# --- 6. Create Final Amazon DataFrame (Write directly to CSV) ---
# Scan all D2(Amazon Reviews) reviews, select only those corresponding to df_master_books, and save
print(f"\n--- Step 6: Scanning D2(Amazon Reviews) and saving to CSV -> {FINAL_AMAZON_CSV} ---")
is_first_chunk = True
if master_isbn_set_final:
    d2_reviews_iterator = pd.read_json(
        path_d2_reviews, lines=True, encoding='latin-1', chunksize=CHUNK_SIZE
    )
    for i, chunk in enumerate(d2_reviews_iterator):
        print(f"   > Processing D2 Reviews chunk {i+1}...") # [Print progress]
        chunk_filtered = chunk[chunk['asin'].isin(master_isbn_set_final)].copy()

        if not chunk_filtered.empty:
            # Standardize column names
            d2_rating_cols = ['reviewerID', 'asin', 'overall', 'reviewText']
            chunk_ready = chunk_filtered[d2_rating_cols].copy()
            chunk_ready = chunk_ready.rename(columns={
                'asin': 'ISBN', 'reviewerID': 'User-ID', 'overall': 'Book-Rating', 'reviewText': 'Review'
            })

            # Merge reviews and master_books based on ISBN
            df_amazon_chunk_final = pd.merge(
                chunk_ready,
                df_master_books[['ISBN', 'Book-Title', 'Book-Author', 'Publisher']],
                on='ISBN',
                how='inner'
            )
            df_amazon_chunk_final = df_amazon_chunk_final[final_columns]

            # First chunk: write mode creates a new file and includes header
            if is_first_chunk:
                df_amazon_chunk_final.to_csv(FINAL_AMAZON_CSV, mode='w', header=True, index=False, encoding='utf-8')
                is_first_chunk = False

            # Second chunk: append mode adds data only
            else:
                df_amazon_chunk_final.to_csv(FINAL_AMAZON_CSV, mode='a', header=False, index=False, encoding='utf-8')
else:
    print("   > Master book list (ISBN) is empty. Skipping Amazon reviews scan.") # [Print progress]

print(f"   > [df_amazon_final] save complete: {FINAL_AMAZON_CSV}") # [Print progress]

print("\n--- All steps complete ---") # [Print progress]

In [ ]:
# Check dataset (amazon, goodreads) info

df_amazon_final = pd.read_csv(
    '/content/drive/MyDrive/data/df_amazon_final.csv',
    encoding='latin-1'
)

print(df_amazon_final.head())
print(df_amazon_final.info())

df_goodreads_book_authors = pd.read_csv(
    '/content/drive/MyDrive/data/df_goodreads_final.csv',
    encoding='latin-1'
)

print(df_goodreads_book_authors.head())
print(df_goodreads_book_authors.info())

# Handling Missing Values

In [ ]:
import pandas as pd

# Load data

df_amazon_final = pd.read_csv(
    '/content/drive/MyDrive/data/df_amazon_final.csv',
    encoding='latin-1'
)
print(df_amazon_final.info())

df_goodreads_final = pd.read_csv(
    '/content/drive/MyDrive/data/df_goodreads_final.csv',
    encoding='latin-1'
)
print(df_goodreads_final.info())


# Remove missing values
# Allow empty Publisher
# Drop rows where the 'Review' column has NaN
df_amazon_final = df_amazon_final.dropna(subset=['Review'])
df_goodreads_final = df_goodreads_final.dropna(subset=['Review'])

print("Amazon (After removal):")
print(df_amazon_final.info())

print("\nGoodreads (After removal):")
print(df_goodreads_final.info())

# EDA

In [ ]:
# D1(goodreads)
total_ratings_d1 = len(df_goodreads_final)
unique_users_d1 = df_goodreads_final['User-ID'].nunique()
unique_books_d1 = df_goodreads_final['ISBN'].nunique()

# D2(amazon)
total_ratings_d2 = len(df_amazon_final)
unique_users_d2 = df_amazon_final['User-ID'].nunique()
unique_books_d2 = df_amazon_final['ISBN'].nunique()

print("[Goodreads]")
print(f"  Total ratings: {total_ratings_d1}")
print(f"  Unique users: {unique_users_d1}")
print(f"  Unique books: {unique_books_d1}")
print(f"  -> Avg ratings per user: {total_ratings_d1 / unique_users_d1:.2f}")
print(f"  -> Avg ratings per book: {total_ratings_d1 / unique_books_d1:.2f}")


print("\n[Amazon Product]")
print(f"  Total ratings: {total_ratings_d2}")
print(f"  Unique users: {unique_users_d2}")
print(f"  Unique books: {unique_books_d2}")
print(f"  -> Avg ratings per user: {total_ratings_d2 / unique_users_d2:.2f}")
print(f"  -> Avg ratings per book: {total_ratings_d2 / unique_books_d2:.2f}")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import warnings
from sklearn.feature_extraction.text import CountVectorizer

# Ignore warning messages
warnings.filterwarnings('ignore')


# Long-Tail Distribution

print("\n--- Long-Tail Distribution ---")

gr_user_counts = df_goodreads_final['User-ID'].value_counts()
gr_item_counts = df_goodreads_final['ISBN'].value_counts()
amz_user_counts = df_amazon_final['User-ID'].value_counts()
amz_item_counts = df_amazon_final['ISBN'].value_counts()

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Long-Tail Distribution Visualization (Log Scale)', fontsize=16)

# (D1) Goodreads Users
sns.histplot(gr_user_counts, ax=axes[0, 0], bins=50, log_scale=True)
axes[0, 0].set_title('[D1] Goodreads User Rating Counts (Log Scale)')
axes[0, 0].set_xlabel('Number of Ratings')

# (D1) Goodreads Books
sns.histplot(gr_item_counts, ax=axes[0, 1], bins=50, log_scale=True)
axes[0, 1].set_title('[D1] Goodreads Book Rating Counts (Log Scale)')
axes[0, 1].set_xlabel('Number of Ratings')

# (D2) Amazon Users
sns.histplot(amz_user_counts, ax=axes[1, 0], bins=50, log_scale=True)
axes[1, 0].set_title('[D2] Amazon User Rating Counts (Log Scale)')
axes[1, 0].set_xlabel('Number of Ratings')

# (D2) Amazon Books
sns.histplot(amz_item_counts, ax=axes[1, 1], bins=50, log_scale=True)
axes[1, 1].set_title('[D2] Amazon Book Rating Counts (Log Scale)')
axes[1, 1].set_xlabel('Number of Ratings')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()



# Popularity Bias

print("\n--- Popularity Bias ---")
gr_item_stats = df_goodreads_final[df_goodreads_final['Book-Rating'] > 0].groupby('ISBN')['Book-Rating'].agg(['count', 'mean']).reset_index()
gr_item_stats.columns = ['ISBN', 'rating_count', 'mean_rating']

# Amazon (D2)
amz_item_stats = df_amazon_final.groupby('ISBN')['Book-Rating'].agg(['count', 'mean']).reset_index()
amz_item_stats.columns = ['ISBN', 'rating_count', 'mean_rating']

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(18, 6))
fig.suptitle('Popularity Bias', fontsize=16)

# (D1) Goodreads
sns.scatterplot(x='rating_count', y='mean_rating', data=gr_item_stats, ax=axes[0], alpha=0.1, s=10)
axes[0].set_title('[D1] Goodreads')
axes[0].set_xlabel('Rating Count')
axes[0].set_ylabel('Mean Rating')
axes[0].set_ylim(0.8, 5.2)
axes[0].set_xlim(None, 1000)

# (D2) Amazon
sns.scatterplot(x='rating_count', y='mean_rating', data=amz_item_stats, ax=axes[1], alpha=0.1, s=10)
axes[1].set_title('[D2] Amazon')
axes[1].set_xlabel('Rating Count')
axes[1].set_ylabel('Mean Rating')
axes[1].set_ylim(0.8, 5.2)
axes[1].set_xlim(None, 500)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


print("\n--- Sentiment Comparison Top Keywords (2-Gram) ---")

# Sample data for efficiency
SAMPLE_SIZE = 50000
gr_sample = df_goodreads_final.sample(n=SAMPLE_SIZE, replace=True, random_state=42)
amz_sample = df_amazon_final.sample(n=SAMPLE_SIZE, replace=True, random_state=42)

# Exclude 0-ratings
gr_sample = gr_sample[gr_sample['Book-Rating'] > 0]
amz_sample = amz_sample[amz_sample['Book-Rating'] > 0]


# Prepare text data (Positive vs. Negative)
# D1: 4, 5 = Positive / 1, 2 = Negative
gr_pos_text_list = gr_sample[gr_sample['Book-Rating'].isin([4, 5])]['Review'].dropna().tolist()
gr_neg_text_list = gr_sample[gr_sample['Book-Rating'].isin([1, 2])]['Review'].dropna().tolist()

# D2: 4, 5 = Positive / 1, 2 = Negative
amz_pos_text_list = amz_sample[amz_sample['Book-Rating'].isin([4, 5])]['Review'].dropna().tolist()
amz_neg_text_list = amz_sample[amz_sample['Book-Rating'].isin([1, 2])]['Review'].dropna().tolist()

# Stopwords
stopwords = set(STOPWORDS)
stopwords.update([
    "book", "read", "books", "reading", "one", "like", "even", "would",
    "really", "story", "get", "much", "Amazon", "Goodreads", "review",
    "author", "title", "publisher", "ISBN"
])

# Helper function: Get Top-N Bigrams
def get_top_n_bigrams(corpus, n=20, custom_stopwords=None):
    if not corpus: # If corpus is empty
        return pd.DataFrame(columns=['bigram', 'count'])

    try:
        # CountVectorizer: Get bigram frequencies
        vec = CountVectorizer(stop_words=list(custom_stopwords),
                                max_features=n, # Top N features
                                ngram_range=(2, 2) # Bigrams only
                               ).fit(corpus)

        # Create bag-of-words
        bag_of_words = vec.transform(corpus)
        # Sum frequencies
        sum_words = bag_of_words.sum(axis=0)
        # Map words to frequencies
        bigrams_freq = [(bigram, sum_words[0, idx]) for bigram, idx in vec.vocabulary_.items()]
        # Sort by frequency
        bigrams_freq = sorted(bigrams_freq, key = lambda x: x[1], reverse=True)

        return pd.DataFrame(bigrams_freq, columns=['bigram', 'count'])

    except ValueError: # e.g., corpus is all stopwords
        return pd.DataFrame(columns=['bigram', 'count'])

# Calculate and print results
print("\n--- [D1] Goodreads Top 20 Bigrams ---")
print("\n[D1] Positive (4-5 Stars):")
# Call the helper function
print(get_top_n_bigrams(gr_pos_text_list, n=20, custom_stopwords=stopwords))
print("\n[D1] Negative (1-2 Stars):")
print(get_top_n_bigrams(gr_neg_text_list, n=20, custom_stopwords=stopwords))

print("\n--- [D2] Amazon Top 20 Bigrams ---")
print("\n[D2] Positive (4-5 Stars):")
print(get_top_n_bigrams(amz_pos_text_list, n=20, custom_stopwords=stopwords))
print("\n[D2] Negative (1-2 Stars):")
print(get_top_n_bigrams(amz_neg_text_list, n=20, custom_stopwords=stopwords))

In [ ]:
df_goodreads_final.to_csv(FINAL_GOODREADS_CSV, index=False, encoding='utf-8')
df_amazon_final.to_csv(FINAL_AMAZON_CSV, index=False, encoding='utf-8')